<a href="https://colab.research.google.com/github/massirism/Codes/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.__version__

'1.6.0+cu101'

In [ ]:
#Creando un tensor
x=torch.tensor([5.5, 3]) #Entre corchetes y separados por una coma se ingresan los datos de las dimensiones

y=torch.tensor([2,4])

#La suma es termino a termino es decir 5.5+2 y 3+4
x+y

#Crea un tensor de unos, las primeras dos variables son las dimensiones del tensor
x=torch.ones(2,2, requires_grad=True) #Si requires__grad es TRUE significa que si se modifica x las variables que realicen operaciones con el cambiarán también
print (x)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [ ]:
#Crear red neuronal con Pytorch, es necesario utilizar la librería torch.nn

#nn.Module contiene capas y forward(input) va a darte la salida 
#Peso =Peso -(lr)*(gradiente), lr se tiene que ajustar

import torch
import torch.nn as nn #importo nuronal network(nn) y guardo como nn
import torch.nn.functional as F #Lo mismo pero con funcional 


class Net(nn.Module): #Defino la clase NET heredera de nn

    #Inicializando la clase
    def __init__(self): 
        super(Net, self).__init__()
        #Red convolucional
        # Datos de entrada: 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3) 
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): #X entrada
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) #Activacion Relu
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#La siguiente función puede ser remplazada por X.flatten(start_dim=1).size()
    def num_flat_features(self, x): #Convierte el tipo de dato (x) pasando la matriz 3*3*3 a un vector de 27, osea aplanar...
 
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
net.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [ ]:
entrada = torch.randn(1, 1, 32, 32) #Cantidad de objetos a ingresar, Un canal de información tamaño ancho y alto 32*32
#Una matriz random que simula imagen de entrada
salida = net(entrada)
salida

tensor([[-0.0539,  0.0200, -0.0415, -0.0049,  0.0726,  0.0812,  0.0133,  0.0618,
         -0.0029,  0.1075]], grad_fn=<AddmmBackward>)

In [ ]:
net.zero_grad() #Resetea los gradientes
salida.backward(torch.randn(1, 10)) #Le paso un vector de errores

In [ ]:
#FUNCION DE ERROR 
salida=net(entrada)
target = torch.randn(10) #Etiqueta, es un vector de 10 clases
target= target.view(1, -1) #View añadiendo el bacht size 
criterio = nn.MSELoss()

loss = criterio(salida, target)
print (loss)

tensor(1.1762, grad_fn=<MseLossBackward>)


In [ ]:
#Para propagar el error, osea el backward(), hay que resetear gradientes
net.zero_grad()

loss.backward()


In [ ]:
#Actualización de los pesos, descensos del gradiente

#PESO=PESO -LR*GRADIENTE

import torch.optim as optim

#Creas un optimizador
optimizador = optim.SGD(net.parameters(), lr=0.01) #sgd?

#En nuestro ciclo de entrenamiento

optimizador.zero_grad()
salida=net(entrada) #red=net 
loss=criterion(salida, target)
loss.backward()
optimizador.step()